# Querying the Gaia eDR3 database
17 March 2022


***
## The Hertzsprung-Russell Diagram in the Gaia era
For additional information about the ESA Gaia mission, have a look at: https://sci.esa.int/web/gaia and for information specific the the current data released (DR2) see https://www.cosmos.esa.int/web/gaia/data-release-2

Loading a few useful packages. Install them on your local computer if needed, e.g. via `pip install astroquery`

In [ ]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from astroquery.gaia import Gaia

import pandas as pd
import numpy as np
import math as m
from matplotlib import pyplot as plt

### Querying the Gaia DR2 database

This shows you all the tables with Gaia data available on the database (don't worry about reading through!)

In [ ]:
from astroquery.gaia import Gaia
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

Here below we query the Gaia database using *ADQL*, the Astronomy Data Query Language. Do not worry about the details of the format, but focus on the criteria used to make the selection:

- the apparent magnitude in the Gaia G band, `phot_g_mean_mag`
- the parallax itself `gaia.parallax`, expressed in mas (milli arcseconds)
- the relative uncertainty on the parallax, so we consider here only stars with precise (10%) determiations of parallax (distance)

In the example given here we are defining a sample limited by the **apparent** magnitue of the observed stars, `gaia.phot_g_mean_mag < 8`.

In [ ]:
job = Gaia.launch_job_async("SELECT * \
FROM gaiaedr3.gaia_source AS gaia \
WHERE \
gaia.phot_g_mean_mag < 8 AND \
gaia.parallax > 0 AND \
gaia.parallax_error/gaia.parallax <= 0.05;", dump_to_file=True)
print (job)

Load the results into a table, and inspect the content. 

In [ ]:
res = job.get_results()
res

There is of course an enourmous amount of information! For now, just focus on these quantities, and on their combination:

- `'parallax'` in mas (milli arcseconds)
- `'phot_bp_mean_mag'` apparent magnitude in the Gaia Bp band (blue)
- `'phot_rp_mean_mag'` apparent magnitude in the Gaia Rp band (red)
- `'phot_g_mean_mag'` apparent magnitude in the Gaia G band (green), see https://www.cosmos.esa.int/web/gaia/iow_20180316 for more info on the Gaia photometric passbands.

as an exercise: 
1. plot a color-magnitude diagram (CMD), e.g. `Bp-Rp` vs the **absolute** magniture in the `G` band computed using the information you have on the parallax. Consider using a density plot or e.g. `hexbin` to highlight highly populated regions in the diagram.
2. can you recognise on the CMD any features you already know? Can you roughly place the Sun on this diagram?
3. explore CMDs resulting from different queries. A suggestion: how and why does the CMD change if you plot a distance-limited sample (e.g. all stars within 50pc) as opposed to a magnitude-limited sample?

**Warning** be careful about the selection criteria in the query: in the database there are \~1.5 billion sources down to magnitude G\~21!!!!

## Worked out example

Remember the relation between apparent magnitude, absolute magnitude and distance or parallax

$m=M +5\log{d}-5$

where d is the distance in pc. From the parallax ($\varpi$, in mas) we can infer a distance (pc):

$d=1000/\varpi$

In [ ]:
abs_mag=res['phot_g_mean_mag']-5*np.log10(1000./res['parallax'])+5;
col=res['phot_bp_mean_mag']-res['phot_rp_mean_mag'];
fig = plt.figure(figsize=(9, 7))
#plt.scatter(col,abs_mag,10)
plt.scatter(col,abs_mag,4, alpha=0.01)

plt.ylabel('$M_{G}$')
plt.xlabel('$m_{Bp}-m_{Rp}$')
plt.ylim([-5, 7.5])
plt.xlim([-0.5, 4])
plt.gca().invert_yaxis()


### Density plot

In [ ]:
fig = plt.figure(figsize=(9, 7))
plt.hexbin(col, abs_mag, cmap='Blues',bins='log')
plt.ylim([-5, 7.5])
plt.xlim([-0.5, 4])
plt.ylabel('$M_{G}$')
plt.xlabel('$m_{Bp}-m_{Rp}$')
plt.gca().invert_yaxis()
cbar = plt.colorbar(extend='both')


### Changing the selction function
Now we search for stars within a given distance from the Sun (50 pc)

In [ ]:
job = Gaia.launch_job_async("SELECT * \
FROM gaiaedr3.gaia_source AS gaia \
WHERE \
gaia.phot_g_mean_mag < 16 AND \
gaia.parallax > 20 AND \
gaia.parallax_error/gaia.parallax <= 0.10;", dump_to_file=False)

In [ ]:
res2 = job.get_results()
len(res2)


In [ ]:
abs_mag2=res2['phot_g_mean_mag']-5*np.log10(1000./res2['parallax'])+5;
col2=res2['phot_bp_mean_mag']-res2['phot_rp_mean_mag'];
fig = plt.figure(figsize=(9, 7))
plt.hexbin(col2, abs_mag2, cmap='Blues',bins='log')
#plt.ylim([-5, 10.5])
#plt.xlim([-0.5, 4])
plt.ylabel('$M_{G}$')
plt.xlabel('$m_{Bp}-m_{Rp}$')
plt.gca().invert_yaxis()
cbar = plt.colorbar(extend='both')



##  Misc

convert an astropy table into pandas dataframe

In [ ]:
df=res2.to_pandas()